In [24]:
import sys
sys.path.append('../SC_models')

from CBD import cbd
import numpy as np
import pandas as pd
import pickle
import dill
import os

import plotly.express as px
import plotly.graph_objects as go
from plotly.validators.scatter.marker import SymbolValidator
import plotly.io as pio

import gif
pio.templates.default = "simple_white"
colors = px.colors.qualitative.Plotly

In [25]:
# Create path for exporting fit plots
path = '../Plots/LBC animations/'
if not os.path.exists(path):
    os.makedirs(path)

In [26]:
# load fitted trajectories stored in a pickle file in 'model_list'
infile = open('../Resources/neutral_fit.pkl','rb')
neutral_fit = pickle.load(infile)
infile.close()

NGF_summary = pd.read_csv('../Resources/NGF_mutation_summary.csv')
#NGF_summary['mutation'] = NGF_summary.apply(lambda x: x['mutation'].split()[0], axis=1)

# Deterministic evolution

In [27]:
def exponential_growth(origin, fitness, tmax=90):
    checkpoints = np.zeros((2, int(tmax)+1))
    checkpoints[0] = np.linspace(0, int(tmax),
                                     int(tmax)+1)
    checkpoints[1] = np.floor(np.exp((checkpoints[0]-origin)*fitness))
    return checkpoints

In [28]:
def part_animation(part_id):

    # Extract data associated to participant
    data = neutral_fit[part_id][0]
    new_data = data[data['error'] == min(data['error'])]
    optimal_cells = new_data.iloc[0].Cells
    new_data = new_data[new_data['Cells'] == optimal_cells]

    NGF_summary_part = NGF_summary[NGF_summary.participant == part_id]

    # Compute the deterministic evolution of clones in part
    clone_list = np.zeros((new_data.shape[0], 2, 91))
    for i, row in enumerate(new_data.iterrows()):
        clone_list[i, :, :] = exponential_growth(row[1].Origin, row[1].Fitness)

    # Compute VAF evolution of clones 
    total_cells = optimal_cells + clone_list[:,1,:].sum(axis=0)
    clone_list[:, 1, :] = clone_list[:, 1, : ] / (2*total_cells)

    # Create melted dataframe
    df = pd.DataFrame (columns =['mutation', 'year', 'VAF'])
    for i in range(clone_list.shape[0]):
        df_2= pd.DataFrame({'mutation': new_data.iloc[i]['Gene name'],#.split()[0],
              'year': clone_list[i,0,:],
              'VAF': clone_list[i,1,:]})
        df = df.append(df_2, ignore_index=True)
    
    animation_start_time = max(0,(clone_list[:,1]!=0).argmax(axis=1).min()-10)
    df = df[df['year']>animation_start_time]


    # create color dictionary
    color_dict = dict()
    for i, mutation in enumerate(NGF_summary_part.mutation.unique()):
        color_dict[mutation] = colors[i]


    # Create animation
    frame_dict = []
    fig = go.Figure()
    n_mutations = len(df.mutation.unique())
    for mutation in NGF_summary_part.mutation.unique():
        mutation_df = NGF_summary_part[NGF_summary_part.mutation == mutation]
        fig.add_trace(go.Scatter(x=mutation_df.year,
                                 y=mutation_df.vaf,
                                 mode='markers',
                                 marker=dict(color=color_dict[mutation],
                                             symbol=SymbolValidator().values[3],
                                             size=10)
                                 ))


    for mutation in df.mutation.unique():
        df_mut = df[df.mutation == mutation]
        fig.add_trace(go.Scatter(x=df_mut['year'][:2],
                                 y=df_mut['VAF'][:2],
                                 name=df_mut['mutation'].unique()[0],
                         mode='lines',
                         line=dict(color=color_dict[mutation], width=3)))

    frames = [dict(data= [dict(type='scatter',
                               x=df[df.mutation == mut]['year'][:k+1],
                               y=df[df.mutation == mut]['VAF'][:k+1])
                          for mut in df.mutation.unique()],
                   traces = list(range(n_mutations, 2*n_mutations )))
              for k in range (1, len(df_mut))]

    fig.layout = go.Layout(
                    updatemenus=[
                        dict(type='buttons',showactive=True,
                                pad=dict(t=0, r=10),
                                buttons=[dict(label='Play',
                                method='animate',
                                args = [None,
                                        {"frame": {"duration": 1/(df.year.max()- df.year.min())*2_000, 
                                                   "redraw": False},
                                                   "fromcurrent": True, 
                                                   "transition": {"duration":0}}]
                                )]
                            )])
    fig.update_layout(xaxis=dict(range=[min(df['year']), max(df['year'])], autorange=False),
                      yaxis =dict(range=[-0.05, NGF_summary_part.vaf.max()+0.05],autorange=False))
    fig.frames=frames
    return fig

In [29]:
part_animation('LBC360636')

In [34]:
@gif.frame
def print_frame(summary, df,  frame_number, color_dict):  
    fig = go.Figure()
    n_mutations = len(df.mutation.unique())
    for mutation in summary.mutation.unique():
        mutation_df = summary[summary.mutation == mutation]
        fig.add_trace(go.Scatter(x=mutation_df.year,
                                 y=mutation_df.vaf,
                                 mode='markers',
                                 marker=dict(color=color_dict[mutation],
                                             symbol=SymbolValidator().values[3],
                                             size=10)
                                 ))


    for mutation in df.mutation.unique():
        df_mut = df[df.mutation == mutation]
        fig.add_trace(go.Scatter(x=df_mut['year'][:frame_number],
                                 y=df_mut['VAF'][:frame_number],
                                 name=df_mut['mutation'].unique()[0],
                         mode='lines',
                         line=dict(color=color_dict[mutation], width=3))) 
        
    fig.update_layout(xaxis=dict(range=[min(df['year']), max(df['year'])], autorange=False),
                      yaxis =dict(range=[-0.05, summary.vaf.max()+0.05],autorange=False))
    return fig

def part_gif (part_id):
    # Extract data associated to participant
    data = neutral_fit[part_id][0]
    new_data = data[data['error'] == min(data['error'])]
    optimal_cells = new_data.iloc[0].Cells
    new_data = new_data[new_data['Cells'] == optimal_cells]

    NGF_summary_part = NGF_summary[NGF_summary.participant == part_id]

    # Compute the deterministic evolution of clones in part
    clone_list = np.zeros((new_data.shape[0], 2, 91))
    for i, row in enumerate(new_data.iterrows()):
        clone_list[i, :, :] = exponential_growth(row[1].Origin, row[1].Fitness)

    # Compute VAF evolution of clones 
    total_cells = optimal_cells + clone_list[:,1,:].sum(axis=0)
    clone_list[:, 1, :] = clone_list[:, 1, : ] / (2*total_cells)

    # Create melted dataframe
    df = pd.DataFrame (columns =['mutation', 'year', 'VAF'])
    for i in range(clone_list.shape[0]):
        df_2= pd.DataFrame({'mutation': new_data.iloc[i]['Gene name'],#.split()[0],
              'year': clone_list[i,0,:],
              'VAF': clone_list[i,1,:]})
        df = df.append(df_2, ignore_index=True)
    
    animation_start_time = max(0,(clone_list[:,1]!=0).argmax(axis=1).min()-5)
    df = df[df['year']>animation_start_time]

    # create color dictionary
    color_dict = dict()
    for i, mutation in enumerate(NGF_summary_part.mutation.unique()):
        color_dict[mutation] = colors[i]

    df_mut=df[df.mutation == df.mutation.unique()[0]]
    gif = [print_frame(NGF_summary_part, df, k, color_dict) for k in range (1, len(df_mut))]

    return gif

In [ ]:
for part in neutral_fit.keys():
    example_gif = part_gif(part)
    gif.save(example_gif, path + f'{part}.gif', duration=100)